In [14]:
import os
import pandas as pd
from sklearn.model_selection import KFold
from collections import defaultdict

# Load the data
current_dir = os.getcwd() 
relative_path_train = os.path.join('..', 'data', 'preprocessed_train_data.csv')
preprocessed_train_data = pd.read_csv(os.path.join(current_dir, relative_path_train))

# Encode target variable
preprocessed_train_data["satisfaction"] = preprocessed_train_data["satisfaction"].replace({"satisfied": 1, "neutral or dissatisfied": 0})



In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import numpy as np
# Step 1: Read the CSV file
df = pd.DataFrame(preprocessed_train_data, columns=["Age", "Flight Distance", "Inflight wifi service", "Ease of Online booking", "Food and drink", "Online boarding", "Seat comfort", "Inflight entertainment", "On-board service", "Leg room service", "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", "Departure Delay in Minutes", "satisfaction"])
X = df.drop("satisfaction", axis=1)  # Features
y = df["satisfaction"]                # Target variable



In [16]:
# Step 1: Mapping
def mapper(data):
    class_counts = {}
    feature_counts = {}
    total_samples = len(data)
    
    for index, row in data.iterrows():
        label = row["satisfaction"]
        #print (label)
        if label not in class_counts:
            class_counts[label] = 1
        else:
            class_counts[label] += 1
        for feature_name, feature_value in row.items():
            if feature_name != "satisfaction":
                key = (label, feature_name, feature_value)
                #print (key)
                if key not in feature_counts:
                    feature_counts[key] = 1
                else:
                    feature_counts[key] += 1
                    
    return class_counts, feature_counts

# Step 2: Reducing
def reducer(mapped_results):
    class_counts = {}
    feature_counts = {}
    total_samples = 0
    
    for c_count, f_count in mapped_results:
        for label, count in c_count.items():
            if label not in class_counts:
                class_counts[label] = count
            else:
                class_counts[label] += count
            total_samples += count
            
        for key, count in f_count.items():
            if key not in feature_counts:
                feature_counts[key] = count
            else:
                feature_counts[key] += count
    #print ("cla count",feature_counts)        
    return class_counts, feature_counts
# Step 3: Train the Naive Bayes classifier
def train_naive_bayes(class_counts, feature_counts):
    class_probabilities = {}
    feature_probabilities = {}

    total_samples = sum(class_counts.values())

    # Calculate class probabilities
    for label, count in class_counts.items():
        class_probabilities[label] = count / total_samples

    # Calculate feature probabilities
    for (label, feature, value), count in feature_counts.items():
        if label not in feature_probabilities:
            feature_probabilities[label] = {}
        if feature not in feature_probabilities[label]:
            feature_probabilities[label][feature] = {}
        feature_probabilities[label][feature][value] = count / class_counts[label]

    return class_probabilities, feature_probabilities

# Step 4: Make predictions
def predict_naive_bayes(class_probabilities, feature_probabilities, sample):
    max_prob = -1
    predicted_class = None

    for label, class_prob in class_probabilities.items():
        prob = class_prob
        for feature, value in sample.items():
            if feature in feature_probabilities[label] and value in feature_probabilities[label][feature]:
                prob *= feature_probabilities[label][feature][value]
            else:
                # Smoothing: Handle unseen feature values
                prob *= 1 / (class_prob + 1)

        if prob > max_prob:
            max_prob = prob
            predicted_class = label

    return predicted_class
# Step 5: Apply map-reduce
class_counts, feature_counts = mapper(df)
final_class_counts, final_feature_counts = reducer([(class_counts, feature_counts)])

# Step 6: Train the Naive Bayes classifier
class_probabilities, feature_probabilities = train_naive_bayes(final_class_counts, final_feature_counts)

# Step 7: Make predictions
predictions = []
for index, row in X.iterrows():
    sample = row.to_dict()
    predicted_class = predict_naive_bayes(class_probabilities, feature_probabilities, sample)
    predictions.append(predicted_class)

# Step 8: Evaluate the classifier
accuracy = accuracy_score(y, predictions)
print("Accuracy:", accuracy)


cla count {(0.0, 'Age', -1.7452792963959505): 535, (0.0, 'Flight Distance', -0.7315387605182878): 103, (0.0, 'Inflight wifi service', 0.2035789516551841): 19386, (0.0, 'Ease of Online booking', 0.173776143853014): 16912, (0.0, 'Food and drink', 1.3522644074658623): 10063, (0.0, 'Online boarding', -0.1855315969806735): 18845, (0.0, 'Seat comfort', 1.1830993068759184): 9259, (0.0, 'Inflight entertainment', 1.2317042817924029): 8864, (0.0, 'On-board service', 0.4794025816766604): 14296, (0.0, 'Leg room service', -0.266840435197607): 14623, (0.0, 'Baggage handling', 0.3117685473283496): 19405, (0.0, 'Checkin service', 0.5497985959374537): 15692, (0.0, 'Inflight service', 1.156435816064447): 10576, (0.0, 'Cleanliness', 1.3058697266087202): 8876, (0.0, 'Departure Delay in Minutes', 0.2663926454868473): 325, (0.0, 'Age', -0.951360238142684): 1868, (0.0, 'Flight Distance', -0.9571835445251908): 90, (0.0, 'Food and drink', -1.6563259160426531): 10269, (0.0, 'Seat comfort', -1.8493145339197603):

In [24]:
import pandas as pd
import numpy as np

# Step 1: Mapping
def mapper(data):
    class_counts = {}
    feature_counts = {}
    total_samples = len(data)
    
    for index, row in data.iterrows():
        label = row["satisfaction"]
        if label not in class_counts:
            class_counts[label] = 1
        else:
            class_counts[label] += 1
        for feature_name, feature_value in row.items():
            if feature_name != "satisfaction":
                key = (label, feature_name, feature_value)
                if key not in feature_counts:
                    feature_counts[key] = 1
                else:
                    feature_counts[key] += 1
                    
    return [(class_counts, feature_counts)]

# Step 2: Reducing
def reducer(mapped_results):
    class_counts = {}
    feature_counts = {}
    total_samples = 0
    
    for c_count, f_count in mapped_results:
        for label, count in c_count.items():
            if label not in class_counts:
                class_counts[label] = count
            else:
                class_counts[label] += count
            total_samples += count
            
        for key, count in f_count.items():
            if key not in feature_counts:
                feature_counts[key] = count
            else:
                feature_counts[key] += count
    return (class_counts, feature_counts)

# Step 3: Train the Naive Bayes classifier
def train_naive_bayes(class_counts, feature_counts):
    class_probabilities = {}
    feature_probabilities = {}

    total_samples = sum(class_counts.values())

    # Calculate class probabilities
    for label, count in class_counts.items():
        class_probabilities[label] = count / total_samples

    # Calculate feature probabilities
    for (label, feature, value), count in feature_counts.items():
        if label not in feature_probabilities:
            feature_probabilities[label] = {}
        if feature not in feature_probabilities[label]:
            feature_probabilities[label][feature] = {}
        feature_probabilities[label][feature][value] = count / class_counts[label]

    return class_probabilities, feature_probabilities

# Step 4: Make predictions
def predict_naive_bayes(class_probabilities, feature_probabilities, sample):
    max_prob = -1
    predicted_class = None

    for label, class_prob in class_probabilities.items():
        prob = class_prob
        for feature, value in sample.items():
            if feature in feature_probabilities[label] and value in feature_probabilities[label][feature]:
                prob *= feature_probabilities[label][feature][value]
            else:
                # Smoothing: Handle unseen feature values
                prob *= 1 / (class_prob + 1)

        if prob > max_prob:
            max_prob = prob
            predicted_class = label

    return predicted_class

# Step 5: Apply map-reduce
# In practice, you would distribute the data across multiple mappers, then reduce the results.
# For simplicity, we'll emulate the map-reduce process with a single mapper and reducer.
mapped_results = mapper(df)
final_class_counts, final_feature_counts = reducer(mapped_results)

# Step 6: Train the Naive Bayes classifier
class_probabilities, feature_probabilities = train_naive_bayes(final_class_counts, final_feature_counts)

# Step 7: Make predictions
predictions = []
for index, row in X.iterrows():
    sample = row.to_dict()
    predicted_class = predict_naive_bayes(class_probabilities, feature_probabilities, sample)
    predictions.append(predicted_class)

# Step 8: Evaluate the classifier
accuracy = accuracy_score(y, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.8355404989220819
